In [23]:
import pandas as pd
import numpy as np
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [12]:
df=pd.read_csv('train.csv')

In [13]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [15]:
df_clean=pd.get_dummies(df[['Survived','Pclass','Sex','Age','Fare','Embarked']],drop_first=True)
df_clean

,Survived,Pclass,Age,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,7.2500,1,0,1
1,1,1,38.0,71.2833,0,0,0
2,1,3,26.0,7.9250,0,0,1
3,1,1,35.0,53.1000,0,0,1
4,0,3,35.0,8.0500,1,0,1
...,...,...,...,...,...,...,...
886,0,2,27.0,13.0000,1,0,1
887,1,1,19.0,30.0000,0,0,1
888,0,3,NaN,23.4500,0,0,1
889,1,1,26.0,30.0000,1,0,0


In [16]:
df_clean.isnull().sum()

Survived        0
Pclass          0
Age           177
Fare            0
Sex_male        0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [17]:
df['Survived'].value_counts()/df.shape[0]

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [18]:
df_clean['Age'].fillna(df['Age'].median(),inplace=True)

In [19]:
df_clean.head()

,Survived,Pclass,Age,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,22.0,7.2500,1,0,1
1,1,1,38.0,71.2833,0,0,0
2,1,3,26.0,7.9250,0,0,1
3,1,1,35.0,53.1000,0,0,1
4,0,3,35.0,8.0500,1,0,1


In [24]:
X_train, X_test, y_train, y_test =train_test_split(df_clean[['Pclass','Sex_male','Age','Fare','Embarked_Q','Embarked_S']],\
                                                   df_clean['Survived'],test_size=0.20,random_state=1)

## Ada Boost CV

In [27]:
from sklearn.ensemble import AdaBoostClassifier

In [52]:
ada_boost=AdaBoostClassifier(random_state=1)
params={'n_estimators':[100,50],
       'learning_rate':[0.001,0.01,0.1,1]}

#n_estimators= The maximum number of estimators at which boosting is terminated.Lower value underfits, higher value may opverfit

#learning_rate= Weight applied to each classifier at each boosting iteration. A higher learning rate increases the 
#               contribution of each classifier. goes from -->(0.0, inf). Smaller value slows learning and prevents overfititng


In [53]:
%%time
mod_cv=GridSearchCV(estimator=ada_boost,
            param_grid=params,
            scoring='accuracy',
            cv=3,
            verbose=1)
mod_cv.fit(X_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Wall time: 2.57 s


GridSearchCV(cv=3, estimator=AdaBoostClassifier(random_state=1),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 1],
                         'n_estimators': [100, 50]},
             scoring='accuracy', verbose=1)

In [54]:
mod_cv.best_params_,mod_cv.best_score_

({'learning_rate': 1, 'n_estimators': 50}, 0.8019714214799843)

In [55]:
pd.DataFrame(mod_cv.cv_results_).iloc[:,6:]

,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,"{'learning_rate': 0.001, 'n_estimators': 100}",0.752101,0.814346,0.801688,0.789378,0.026861,2
1,"{'learning_rate': 0.001, 'n_estimators': 50}",0.752101,0.814346,0.801688,0.789378,0.026861,2
2,"{'learning_rate': 0.01, 'n_estimators': 100}",0.752101,0.814346,0.801688,0.789378,0.026861,2
3,"{'learning_rate': 0.01, 'n_estimators': 50}",0.752101,0.814346,0.801688,0.789378,0.026861,2
4,"{'learning_rate': 0.1, 'n_estimators': 100}",0.764706,0.784810,0.805907,0.785141,0.016822,8
5,"{'learning_rate': 0.1, 'n_estimators': 50}",0.756303,0.789030,0.810127,0.785153,0.022144,7
6,"{'learning_rate': 1, 'n_estimators': 100}",0.798319,0.784810,0.784810,0.789313,0.006368,6
7,"{'learning_rate': 1, 'n_estimators': 50}",0.798319,0.805907,0.801688,0.801971,0.003104,1


In [56]:
best_adaboost=AdaBoostClassifier(learning_rate=5,n_estimators=100)
best_adaboost.fit(X_train,y_train)

AdaBoostClassifier(learning_rate=5, n_estimators=100)

In [57]:
pred=best_adaboost.predict(X_test)

In [58]:
accuracy_score(y_test,pred)

0.776536312849162